Transformaciones

In [1]:
#abrir dataset con pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
data=pd.read_csv("datasets/movies_dataset.csv")
data.info()

C:\Users\santi\AppData\Local\Temp\ipykernel_14656\2754094420.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("movies_dataset.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

1-Eliminamos las columnas que no vamos a utilizar

In [2]:
#Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.
data.drop(['video','imdb_id','adult','original_title','vote_count','poster_path','homepage'],axis=1,inplace=True)

2-desanidamos el json de belong_to_collection


In [4]:
#desanidar el json de belongs_to_collection
import json
df=data
#el formato de la columna belongs_to_collection es un json en este formato {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
# Convertir cadenas JSON en diccionarios Python
# Convertir los valores de la columna "belongs_to_collection" en objetos JSON válidos


def load_json(x):
    try:
        return json.loads(x.replace("'", "\""))
    except (json.JSONDecodeError, AttributeError):
        return {}

# Convertir los valores de la columna "belongs_to_collection" en objetos JSON válidos
df["belongs_to_collection"] = df["belongs_to_collection"].apply(load_json)

# Crear nuevas columnas para cada campo en el objeto JSON
df["collection_id"] = df["belongs_to_collection"].apply(lambda x: x["id"] if isinstance(x, dict) and "id" in x else None)
df["collection_name"] = df["belongs_to_collection"].apply(lambda x: x["name"] if isinstance(x, dict) and "name" in x else None)
df["collection_poster"] = df["belongs_to_collection"].apply(lambda x: x["poster_path"] if isinstance(x, dict) and "poster_path" in x else None)
df["collection_backdrop"] = df["belongs_to_collection"].apply(lambda x: x["backdrop_path"] if isinstance(x, dict) and "backdrop_path" in x else None)
df.drop(['belongs_to_collection'],axis=1,inplace=True)
















0    /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1                                None
2    /hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3                                None
4    /7qwE57OVZmMJChBpLEbJEmzUydk.jpg
5                                None
6                                None
7                                None
8                                None
9    /6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
Name: collection_backdrop, dtype: object

2.1 observamos el resultado:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45466 non-null  object 
 1   genres                45466 non-null  object 
 2   id                    45466 non-null  object 
 3   original_language     45455 non-null  object 
 4   overview              44512 non-null  object 
 5   popularity            45461 non-null  object 
 6   production_companies  45463 non-null  object 
 7   production_countries  45463 non-null  object 
 8   release_date          45379 non-null  object 
 9   revenue               45460 non-null  float64
 10  runtime               45203 non-null  float64
 11  spoken_languages      45460 non-null  object 
 12  status                45379 non-null  object 
 13  tagline               20412 non-null  object 
 14  title                 45460 non-null  object 
 15  vote_average       

3-Observamos que la columna genres tambien tiene los datos anidados, por lo tanto procedemos a realizar lo mismo que con bellong_to_collection

In [7]:
#desanidar el json de genres
import json
#el formato de la columna genres es un json en este formato [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

# Definir la función para cargar los objetos JSON
def load_json(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except:
        return {}

# Aplicar la función a la columna "genres"
df["genres"] = df["genres"].apply(load_json)
df["genre_ids"] = df["genres"].apply(lambda x: [d.get("id") for d in x])
df["genre_names"] = df["genres"].apply(lambda x: [d.get("name") for d in x])
df["genre_names"].head(10)





0           [Animation, Comedy, Family]
1          [Adventure, Fantasy, Family]
2                     [Romance, Comedy]
3              [Comedy, Drama, Romance]
4                              [Comedy]
5      [Action, Crime, Drama, Thriller]
6                     [Comedy, Romance]
7    [Action, Adventure, Drama, Family]
8         [Action, Adventure, Thriller]
9         [Adventure, Action, Thriller]
Name: genre_names, dtype: object

3.1 volvemos a ver los resultados

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                45466 non-null  object 
 1   genres                45466 non-null  object 
 2   id                    45466 non-null  object 
 3   original_language     45455 non-null  object 
 4   overview              44512 non-null  object 
 5   popularity            45461 non-null  object 
 6   production_companies  45463 non-null  object 
 7   production_countries  45463 non-null  object 
 8   release_date          45379 non-null  object 
 9   revenue               45460 non-null  float64
 10  runtime               45203 non-null  float64
 11  spoken_languages      45460 non-null  object 
 12  status                45379 non-null  object 
 13  tagline               20412 non-null  object 
 14  title                 45460 non-null  object 
 15  vote_average       

4-Hacemos el mismo proceso con production_companies y production_countries

In [9]:

df['production_companies'].head(10)
#production_companies es un json en este formato [{'name': 'Pixar Animation Studios', 'id': 3}, {'name': 'Walt Disney Pictures', 'id': 2}]
# Definir la función para cargar los objetos JSON
def load_json(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except:
        return {}

# Aplicar la función a la columna "production_companies"
df["production_companies"] = df["production_companies"].apply(load_json)
df["production_companies_ids"] = df["production_companies"].apply(lambda x: [d.get("id") for d in x])
df["production_companies_names"] = df["production_companies"].apply(lambda x: [d.get("name") for d in x])
df["production_companies_ids"].head(10)


0                                         [3]
1                          [559, 2550, 10201]
2                               [6194, 19464]
3                                       [306]
4                                [5842, 9195]
5                            [508, 675, 6194]
6    [4, 258, 932, 5842, 14941, 55873, 58079]
7                                         [2]
8                          [33, 21437, 23770]
9                                  [60, 7576]
Name: production_companies_ids, dtype: object

In [10]:

import json


def load_json(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except:
        return None
    

# Aplica la función a cada elemento de la columna 'production_countries'
df["country"] = df["production_countries"].apply(lambda x: load_json(x))
df["country"] = df["country"].apply(lambda x: x[0]["name"] if isinstance(x, list) and len(x) > 0 else None)
df["country"].head(10)


0    United States of America
1    United States of America
2    United States of America
3    United States of America
4    United States of America
5    United States of America
6                     Germany
7    United States of America
8    United States of America
9              United Kingdom
Name: country, dtype: object

5-aplicamos lo mismo con spoken_lenguages

In [11]:

import json
def load_json(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except:
        return None

# Aplica la función a cada elemento de la columna 'spoken_languages'
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: load_json(x))
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: x[0]["name"] if isinstance(x, list) and len(x) > 0 else None)
df["spoken_languages"].head(10)

0     English
1     English
2     English
3     English
4     English
5     English
6    Français
7     English
8     English
9     English
Name: spoken_languages, dtype: object

(borramos los datos nulos de la siguientes columnas)

In [12]:
df.drop(['genres','production_companies','production_countries'],axis=1,inplace=True)
df.head(10)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,status,...,vote_average,collection_id,collection_name,collection_poster,collection_backdrop,genre_ids,genre_names,production_companies_ids,production_companies_names,country
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,English,Released,...,7.7,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[16, 35, 10751]","[Animation, Comedy, Family]",[3],[Pixar Animation Studios],United States of America
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,English,Released,...,6.9,NaN,None,None,None,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",United States of America
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,English,Released,...,6.5,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg,"[10749, 35]","[Romance, Comedy]","[6194, 19464]","[Warner Bros., Lancaster Gate]",United States of America
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,English,Released,...,6.1,NaN,None,None,None,"[35, 18, 10749]","[Comedy, Drama, Romance]",[306],[Twentieth Century Fox Film Corporation],United States of America
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,English,Released,...,5.7,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg,[35],[Comedy],"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",United States of America
5,60000000,949,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15,187436818.0,170.0,English,Released,...,7.7,NaN,None,None,None,"[28, 80, 18, 53]","[Action, Crime, Drama, Thriller]","[508, 675, 6194]","[Regency Enterprises, Forward Pass, Warner Bros.]",United States of America
6,58000000,11860,en,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15,0.0,127.0,Français,Released,...,6.2,NaN,None,None,None,"[35, 10749]","[Comedy, Romance]","[4, 258, 932, 5842, 14941, 55873, 58079]","[Paramount Pictures, Scott Rudin Productions, ...",Germany
7,0,45325,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,1995-12-22,0.0,97.0,English,Released,...,5.4,NaN,None,None,None,"[28, 12, 18, 10751]","[Action, Adventure, Drama, Family]",[2],[Walt Disney Pictures],United States of America
8,35000000,9091,en,International action superstar Jean Claude Van...,5.23158,1995-12-22,64350171.0,106.0,English,Released,...,5.5,NaN,None,None,None,"[28, 12, 53]","[Action, Adventure, Thriller]","[33, 21437, 23770]","[Universal Pictures, Imperial Entertainment, S...",United States of America
9,58000000,710,en,James Bond must unmask the mysterious head of ...,14.686036,1995-11-16,352194034.0,130.0,English,Released,...,6.6,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg,"[12, 28, 53]","[Adventure, Action, Thriller]","[60, 7576]","[United Artists, Eon Productions]",United Kingdom


6- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [13]:
#Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0
df['revenue'].fillna(0,inplace=True)
df['budget'].fillna(0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   budget                      45460 non-null  object 
 1   id                          45460 non-null  object 
 2   original_language           45449 non-null  object 
 3   overview                    44506 non-null  object 
 4   popularity                  45460 non-null  object 
 5   release_date                45376 non-null  object 
 6   revenue                     45460 non-null  float64
 7   runtime                     45203 non-null  float64
 8   spoken_languages            41605 non-null  object 
 9   status                      45379 non-null  object 
 10  tagline                     20412 non-null  object 
 11  title                       45460 non-null  object 
 12  vote_average                45460 non-null  float64
 13  collection_id               316

7-Los valores nulos del campo release date deben eliminarse.

In [14]:
#Los valores nulos del campo release date deben eliminarse.
df.dropna(subset=['release_date'],inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   budget                      45376 non-null  object 
 1   id                          45376 non-null  object 
 2   original_language           45365 non-null  object 
 3   overview                    44435 non-null  object 
 4   popularity                  45376 non-null  object 
 5   release_date                45376 non-null  object 
 6   revenue                     45376 non-null  float64
 7   runtime                     45130 non-null  float64
 8   spoken_languages            41582 non-null  object 
 9   status                      45296 non-null  object 
 10  tagline                     20398 non-null  object 
 11  title                       45376 non-null  object 
 12  vote_average                45376 non-null  float64
 13  collection_id               316

8- creamos las columnas release_year y release_month

In [15]:

#deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
df['release_year'] = pd.DatetimeIndex(df['release_date']).year
df['release_year'].head(10)
#crear la columna release_month donde extraerán el mes de la fecha de estreno.
df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df['release_month'].head(10)



0    10
1    12
2    12
3    12
4     2
5    12
6    12
7    12
8    12
9    11
Name: release_month, dtype: int64

9-hacemos las ultimas conversiones y guardamos el dataset limpio en un csv

In [17]:
# convertir los datos de revenue y budget a float
df['revenue'] = df['revenue'].astype(float)
df['budget'] = df['budget'].astype(float)

# crear la columna return donde calcularán la ganancia de la película
df['return'] = df.apply(lambda row: 0 if row['budget'] == 0 else row['revenue']/row['budget'], axis=1)

#convertir la columna release_date a datetime
df['release_date'] = pd.to_datetime(df['release_date'])
#crear un csv con el nombre movies_clean.csv
df.to_csv('movies_clean.csv',index=False)
